In [ ]:
import pandas as pd
import Read_file as rf
import datetime as dt
import Dn_dt as dn
import Tech_Indi as td

In [ ]:
data = rf.Read_xlsx_File.Read_xlsx_File("../Data_File/sym_data/NIFTY.xlsx", int(input("0 = NFITY50\n1 = NIFTY100\n2 = NFITY200\n3 = NIFTY-NEXT50\n4 = Please inter your symbols in Port_folio1 and numbers are continues.5 6 7\n Please enter your Choice: ")))
symbols = rf.Read_xlsx_File.symbols(data)
sym = symbols[:5]

In [ ]:
Tfile = input("0: historical_data,\n1: 5min_data,\nEnter your choice: ")
file_path = "../Data_File/hisdata"
if Tfile == "0":
    file_path = "../Data_File/hisdata/hdata/"
elif Tfile == "1":
    file_path = "../Data_File/hisdata/5min_data"
else:
    pass

In [ ]:
def golden_cross_strategy(data, sma_low, sma_high): # golden cross strategy is not working.
    # Generate signals
    data['Buy'] = 0
    data['Sell'] = 0
    buy_flag = False
    sell_flag = False
    
    for i in range(len(data)):
        if not buy_flag:
            # Check for Golden Cross
            if data[f'SMA_{sma_low}'].iloc[i] > data[f'SMA_{sma_high}'].iloc[i]:
                data.at[data.index[i], 'Buy'] = 1
                buy_flag = True
        else:
            # Check for Death Cross
            if data[f'SMA_{sma_high}'].iloc[i] > data[f'SMA_{sma_low}'].iloc[i]:
                data.at[data.index[i], 'Sell'] = -1
                buy_flag = False
    
    return data

def rsi_strategy(data, periods):
    data['Buy'] = 0
    data['Sell'] = 0
    buy_flag = False
    
    for i in range(len(data)):
        if not buy_flag:
            # Check for Golden Cross
            if data[f'RSI_{period}'].iloc[i] < 35:
                data.at[data.index[i], 'Buy'] = 1
                buy_flag = True
                sell_flag = False
        elif not sell_flag:
            # Check for Death Cross
            if data[f'RSI_{period}'].iloc[i] > 65:
                data.at[data.index[i], 'Sell'] = -1
                buy_flag = False
                sell_flag = True
    
    return data



'''    
    # Buy signal (Golden Cross)
    data.loc[data[f'SMA_{sma_low}'] > data[f'SMA_{sma_high}'], 'Buy'] = 1  
    # Sell signal (Death Cross)
    data.loc[data[f'SMA_{sma_high}'] > data[f'SMA_{sma_low}'], 'Sell'] = -1
    # Sell signal (RSI overbought)
    data.loc[data[f'RSI_{period}'] > 65, 'Sell'] = -1  
    # Buy signal (RSI oversold)
    data.loc[data[f'RSI_{period}'] < 35, 'Buy'] = 1  
    
    return data
'''


In [ ]:
def calculate_wealth(signals, data, wealth):
    # Initialize buy and sell variables
    buy_price = None
    sell_price = None
    in_position = False  # Flag to track if currently in a position (buy)

    # Initialize variables for tracking total gain and number of trades
    total_gain = 0
    num_trades = 0

    # Iterate through signals
    for index, row in signals.iterrows():
        if row['Buy'] == 1 and not in_position:  # Buy signal and not in a position
            if 'Close' in data:  # Check if 'Close' column exists
                buy_price = data.loc[index, 'Close']
                in_position = True  # Set flag to true
        elif row['Sell'] == -1 and in_position:  # Sell signal and in a position
            if 'Close' in data:  # Check if 'Close' column exists
                sell_price = data.loc[index, 'Close']
                gain = (sell_price - buy_price)  # Calculate gain
                total_gain += gain
                num_trades += 1
                # Reset buy and sell prices and position flag
                buy_price = None
                sell_price = None
                in_position = False  # Set flag to false

    # Calculate final wealth
    final_wealth = wealth + total_gain

    # Return total gain and final wealth
    return total_gain, final_wealth


In [ ]:
import matplotlib.pyplot as plt

def plot_signals(dataframe, symbol):
    # Plotting the Close price
    plt.figure(figsize=(12, 6))
    plt.plot(dataframe[symbol].index, dataframe[symbol]['Close'], label='Close Price', color='blue')

    # Plotting Buy signals
    plt.plot(dataframe[symbol][dataframe[symbol]['Buy'] == 1].index,
             dataframe[symbol]['Close'][dataframe[symbol]['Buy'] == 1],
             '^', markersize=10, color='green', lw=0, label='Buy Signal')

    # Plotting Sell signals
    plt.plot(dataframe[symbol][dataframe[symbol]['Sell'] == -1].index,
             dataframe[symbol]['Close'][dataframe[symbol]['Sell'] == -1],
             'v', markersize=10, color='red', lw=0, label='Sell Signal')

    plt.title('Golden Cross and RSI Strategy Signals')
    plt.xlabel('Date')
    plt.ylabel('Close Price')
    plt.legend()
    plt.show()

# Example usage:
# plot_signals(dt5, 'AAPL')  # Assuming 'dt5' is your dataframe and 'AAPL' is the symbol


In [ ]:
wealth = 500
sma_low = 50
sma_high = 200
period = 14
dt5 = {}
for symbol in symbols:
    tech_model = td.Tech_model()
    try:
        # Assuming the file names are in the format 'hisdata/symbol.xlsx'
        file = f"{file_path}/{symbol}.xlsx"
        Hdata = rf.Read_xlsx_File.Read_xlsx_File(file)
        # Do something with Hdata
        #SMA
        low_sma, high_sma = tech_model.SMA(df=Hdata, low_timeperiod=sma_low, high_timeperiod=sma_high)
        #RSI
        rsi = tech_model.RSI(df=Hdata, window=period)
        #Position of Up or Down.
        Hdata['Position'] = Hdata.apply(tech_model.Perpus, axis=1)
        # Dropping empty values and converting to dictionary.
        Hdata.dropna(inplace=True)
        columns = ['Adj Close']
        dt5[symbol] = Hdata
        #Call the function to get signals for each symbol.    
        sig1 = golden_cross_strategy(dt5[symbol], sma_low, sma_high)
        #sig2 = rsi_strategy(dt5[symbol],period)
        total_gain1, final_wealth1 = calculate_wealth(sig1, dt5[symbol], wealth)
        #total_gain2, final_wealth2 = calculate_wealth(sig2, dt5[symbol], wealth)
        print(f"Total gain in wealth by :{symbol} Rs. {total_gain1}, {final_wealth1}")
        #plot_signals(dt5, symbol)
        
    except FileNotFoundError:
        pass
        #print(f"File for {symbol} not found.")

In [ ]:
print(dt5['CIPLA.NS'][f'SMA_{sma_low}'])

In [ ]:
print(dt5['CIPLA.NS'][['SMA_50', 'SMA_200', 'RSI_14']])


In [ ]:
sma_low = 50
sma_high = 200
period = 14

signals = golden_cross_strategy(dt5['CIPLA.NS'], sma_low, sma_high, period)
(dt5)

In [ ]:
hellow

## TODO: 
    RSI and MACD Or Bollinger Bands Strategy.

In [ ]:
sma_low = 10
sma_high = 27
ema_low = 7
ema_high = 15
period = 14
yf_dn_obj = dn.yf_dn.yf_data
dt5 = {}
for symbol in sym:
    data = yf_dn_obj(self = symbol, symbol=symbol)
    #Technical Indicators
    tech_model = td.Tech_model()
    
    #SMA
    low_sma, high_sma = tech_model.SMA(df=data, low_timeperiod=sma_low, high_timeperiod=sma_high)
    #EMA
    low_ema, high_ema = tech_model.EMA(df=data, low_timeperiod=ema_low, high_timeperiod=ema_high)
    #RSI
    rsi = tech_model.RSI(df=data, window=period)
    #Position of Up or Down.
    data['Position'] = data.apply(tech_model.Perpus, axis=1)
    
    # Dropping empty values and converting to dictionary.
    data.dropna(inplace=True)
    columns = ['Adj Close']
    data.drop(columns=columns, inplace=True)
    dt5[symbol] = data
    '''
    # FIXME: Plotting This is working for ploting data with different symbols wise data.
    plt.figure(figsize=(10, 6))
    plt.plot(data.index, data["Close"], label='Close Price', color='blue')
    #plt.plot(data.index, data["Low"], label='Low Price', color='black')
    plt.plot(data.index, data[f'SMA{sma_low}'], label=f'SMA{sma_low}', color='red')
    plt.plot(data.index, data[f'SMA{sma_high}'], label=f'SMA{sma_high}', color='green')
    plt.plot(data.index, data[f'EMA{ema_low}'], label=f'EMA{ema_low}', color='orange')
    plt.plot(data.index, data[f'EMA{ema_high}'], label=f'EMA{ema_high}', color='purple')
    plt.title(f'{symbol} Close Price with SMAs and EMAs')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.show()
    '''


In [ ]:
print (len(dt5['COALINDIA.NS']))